# 가설검정
## 1절 상관분석

In [ ]:
import pandas as pd
from sklearn.datasets import load_diabetes

diabetes=load_diabetes()
data=pd.DataFrame(diabetes.data,columns=diabetes.feature_names)
data.info() # 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [28]:
# scipy.stats.pearsonr
from scipy.stats import pearsonr
pearsonr(x=data['age'],y=data['bmi'])

PearsonRResult(statistic=0.1850846661465555, pvalue=9.076791865417418e-05)

- 피어슨상관계수 (상관계수, p-value) 형태로 출력
- 상관계수가 0.185로 매우 상관관계적음
- p-value 0.05 이하로 귀무가설을 기각. 상관계수는 적지만 유의미하다

In [29]:
# 단순한 상관계수 산출은 corr으로도 가능
data[['sex','bmi']].corr()

,sex,bmi
sex,1.000000,0.088161
bmi,0.088161,1.000000


## 2절 정규성검정

In [30]:
import numpy as np

# 균일표본 생성
np.random.seed(2024)  # 결과의 일관성을 위해 시드 설정
x=np.random.random(10)

# 샤피로-윌크 검정 수행
from scipy.stats import shapiro
shapiro(x)

ShapiroResult(statistic=0.8969663381576538, pvalue=0.20285381376743317)

- (통계량, p-value)형태
- pvalue가 0.05를 넘기에 귀무가설을 기각못함 -> 정규성만족

## 3절 모평균과 모분산 검정

### 모평균 검정
#### 단일 표본 t검정

In [31]:
# Q. 몸무게 데이터를 임의로 생성해보고
# 모집단의 평균이 70이라고 할 수 있는지 단일표본 t-검정을 수행해보자. (정규성만족가정)
import numpy as np

# 몸무게 데이터 임의 생성
kg=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0,50.7,58.4,74.1,65.1,77.8,48.1,46.3])

# 표본평균
np.mean(kg)

66.56428571428572

In [32]:
# 단일표본 t 검정
from scipy.stats import ttest_1samp
ttest_1samp(kg,70)

TtestResult(statistic=-1.0289933120202257, pvalue=0.3222484823978743, df=13)

- (t통계량값, p-value)형태
- p-value가 0.32로 0.05이상이므로 귀무가설을 기각못함 평균은 70과 같다고 할수있다

#### 대응표본 t검정

In [33]:
# Q. 남녀 몸무게 데이터를 임의로 생성하고, 두 데이터가 서로 짝지어져 있다고 가정하고, 
# 두 데이터에 대한 모평균이 서로 다르다고 할 수 있는지 대응표본 t-검정을 수행해보자. (정규성만족가정)
import pandas as pd
female=np.array([50.7,58.4,74.1,65.1,77.8,48.1,46.3])
male=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0])

# 두데이터의 차이의 평균
diff= female-male
np.mean(diff)

-12.985714285714291

In [34]:
from scipy.stats import ttest_rel
ttest_rel(female,male)

TtestResult(statistic=-2.078446933064972, pvalue=0.08291274205610201, df=6)

- (t통계량,p-value)
- p value 가 0.05보다 크므로 유의수준에서 귀무가설을 기각 못함 
=> 둘의 모평균이 다르다고 할수없다

#### 독립표본 t검정

In [35]:
# Q. 위의 몸무게 데이터를 활용해 두 데이터가 독립이라고 가정하고, 
# 두 데이터에 대한 모평균이 서로 다르다고 할 수있는지 독립표본 t-검정을 수행해보자. (정규성, 등분산성 만족 가정)
# 두 데이터가 서로 독립이라고 가정하고 시행
from scipy.stats import ttest_ind
ttest_ind(female,male)

Ttest_indResult(statistic=-2.2186641577772956, pvalue=0.046550122110569664)

- (t통계량, p-value)형태
- p-value 가 유의수준 0.05보다 작으므로 귀무가설 기각 -> 모평균은 서로 다르다고 할수 있다

### 모분산검정
#### 단일표본 검정

In [36]:
# Q. 임의의 점수 데이터를 임의로 생성하고, 모분산이 1,100보다 작다고 할 수 있는 주장의 입증을 위한 가설검정을 수행해보자.(정규성만족 가정)
score=np.array([80.5,60.2,70,87,45,91,85])

# 검정통계량 = (n-1)*표본분산/귀무가설에서 설정한 모분산
var0=1100   # 귀무가설에서 설정한 모분산
var=np.var(score,ddof=1)    # 표본분산
print(var)

278.1033333333333


In [37]:
stat=(len(score)-1)*var/var0 # 검정통계량
print(stat)

1.5169272727272727


In [38]:
# 카이분포를 통해 직접 유의확률 계산
# 좌측검정이므로 Pr[chisq2(자유도)<검정통계량]으로 계산
from scipy.stats import chi2
chi2.cdf(stat,len(score)-1)

0.041637780038918736

- 검정결과 p-value가 0.0416이므로 유의수준 5%에서 귀무가설을 기각하기 떄문에 모분산이 1100보다 작다고 할수 있다.

In [39]:
# (참고) 우측 검정일 경우,  Pr[chisq2(자유도)>검정통계량]으로 계산
1-chi2.cdf(stat,len(score)-1)

0.9583622199610813

In [40]:
# (참고) 양측 검정일 경우, 2*좌측검정의 유의확률 또는 2*우측검정의 유의확률로 계산
# 여기서는 표분분산< 모분산이므로 좌측검정의 유의확률에서 2배를 곱함
2*chi2.cdf(stat,len(score)-1)

0.08327556007783747

In [41]:
# 유의수준 5%에서 모분산이 1100보다 작다고 할 수는 있는 있지만 통계적으로 다르다고 하기는 어렵다.

#### 분산비 검정

In [42]:
# Q. 두 집단 (a,b)에 대한 점수 데이터 임의로 생성하고 a 집단의 모분산이 b집단 모분산보다
# 작다고 할수 있는 지를 검정해보자. (가설은 a 모분산< b모분산으로 가정, 정규성 만족 가정)
# 두 집단(a,b)에 대한 점수 데이터 임의로 생성
a=np.array([70,80,75,65,100,98])
b=np.array([20,100,50,94,28,80,95,30])

# 표본 분산 계산
var_a=np.var(a,ddof=1)
var_b=np.var(b,ddof=1)
print(var_a,var_b)

212.66666666666669 1138.4107142857142


In [43]:
# 검정통계량 = 집단 a의 분산/집단b의 분산(가설: a모분산<b모분산)
stat=var_a/var_b
print(stat)

0.18681014154026346


In [44]:
# 자유도 계산
df1,df2=len(a)-1,len(b)-1
print(df1,df2)

5 7


In [45]:
# F분포를 통해 직접 유의 확률 계산
# 좌측검정이므로 Pr[F(자유도1,자유도2)>검정통계량]으로 계산
# 좌측검정인이유? 가설: a 모분산 < b 모분산 <=> a모분산/b모분산<1)
from scipy.stats import f
pval=f.cdf(stat,df1,df2)
print(pval)

0.04153943037562959


- 검정결과 p-value 가 0.0415 이므로 5% 미만 귀무가설 기각. a모분산 < b모분산 이라고 할수있다/

In [46]:
# (참고) 우측검정일경우, Pr[F(자유도1,자유도2)>검정통계량]으로 계산
1-f.cdf(stat,df1,df2)

0.9584605696243704

In [47]:
# (참고) 양측 검정일 경우, 현재 검정통계량이 보다 작으므로 아래와 같이계산함
# Pr[F(자유도1,자유도2)<검정통계량] + Pr[F( 자유도2,자유도1)>1/검정통계량]
f.cdf(stat,df1,df2)+(1-f.cdf(1/stat,df2,df2))

0.062370657581417376

In [48]:
# 유의수준 5%에서 a 집단의 모분산이 b집단의 모분산보다 작다고 할 수있짐나 통계적으로 다르다고하기는 어렵다.

#### Bartlett 검정
- 정규성을 충족하는 데이터에서 등분산성 검정

In [49]:
# 임의로 세 그룹의 점수 데이터를 생성한 후, 생성된 표본들 간의 등분산성 확인을 위한 Bartlett검정을 수행해보자
# 두 집단(a,b)에 대한 점수 데이터를 임의로 생성
# 임의로 세 그룹의 점수 데이터를 생성
a=np.array([70,80,75,65,100,98])
b=np.array([20,100,50,94,28,80])
c=np.array([90,97,95,94,99,100])

In [50]:
# Bartlett 검정수행
from scipy.stats import bartlett
bartlett(a,b,c)

BartlettResult(statistic=15.6702722148674, pvalue=0.00039558846873743075)

- (검정통계량, p-value)
- p-value가 매우 작으므로 귀무가설을 기각, 그룹간 분산에 유의미한 차이가 있다고 할수있따.

#### levene 검정
- Levene검정은 정규성을 충족하지 않는 데이터에 등분산검정

In [51]:
from scipy.stats import levene
levene(a,b,c)

LeveneResult(statistic=14.365736704446384, pvalue=0.00032713621045500125)

- (검정통계량, p-value)
- p-value가 매우 작으므르 유의수준 5% 에서 귀무가설 기각 <그룹간으 ㅣ분산에 유의미한 차이가 있따

### 분산분석
#### 일원배치 분산분석

In [53]:
#Q. A~C 세개 학교에서 각 1000명씩 층화추출한 데이터를 활용해 
# 세 학교 간 기말고사 성적으 ㅣ평균이 서로 차이가 있는지 검정하고자 한다.
# (성별, 학년의 비율을 세 학교가 동일, 정규성, 등분산성등 만족 가정) 
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [56]:
df=pd.read_csv('../예제/school_score.csv')
print(df)

        id School Sex  Grade  Final
0        1      A   M      1   44.4
1        2      A   M      2   47.7
2        3      A   M      3   65.6
3        4      A   F      1   50.7
4        5      A   F      2   51.3
...    ...    ...  ..    ...    ...
2995  2996      C   M      2   84.1
2996  2997      C   M      3   81.6
2997  2998      C   F      1   83.2
2998  2999      C   F      2   92.8
2999  3000      C   F      3  100.0

[3000 rows x 5 columns]


In [57]:
one_way=ols('Final~C(School)',data=df).fit()

In [58]:
# anova_lm 함수를 사용해 분산분석
result=anova_lm(one_way)
print(result)

               df         sum_sq        mean_sq            F  PR(>F)
C(School)     2.0  996939.237147  498469.618573  5722.221007     0.0
Residual   2997.0  261072.308290      87.111214          NaN     NaN


- C(School)의 Pr(>F) 값이 매우 작으므로 유의수준 5%에서 귀무가설을 기각 
- 세학교 간 기말고사 성적의 평균이 서로 차이가 있다.

#### 사후검정

In [59]:
# Q. 위에서 수행된 분산분석 이후 과정인 사후검정을 수행해보자. (튜키의 다중비교로 진행)
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# tukey의 다중비교 시행
tukey_result=pairwise_tukeyhsd(endog=df['Final'],
                               groups=df['School'],
                               alpha=0.05)
print(tukey_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B  38.8464    0.0 37.8677 39.8251   True
     A      C  38.4922    0.0 37.5135 39.4709   True
     B      C  -0.3542 0.6728 -1.3329  0.6245  False
----------------------------------------------------


- 두 그룹간의 평균차이(meandiff)와 95%신뢰구간(lower,upper)
- 신뢰구간에 0을 포함하지 않는 A,B 와 A,C 간에는 기말고사 평균은 차이가 있고, B와 C는 신뢰구간에 0을 포함하지 않으므로 평균의 차이가 없다고 할수있다.
- reject 결과만으로도 False인경우 유의미한 차이가 없고, True인 경우 유의미한 차이가 있다.


#### 이원배치 분산분석

In [61]:
# Q. 위에서 사용한 데이터를 활용해 학교와 성별이 기말고사 성적이 영향을 주는지 알아보기 위해 이원배치 분산분석을 수행해보자
two_way=ols('Final~C(School)+C(Grade)+C(School):C(Grade)',data=df).fit()

# anova_lm함수를 사용해 이원배치 분산분석
result=anova_lm(two_way)
print(result)

                        df         sum_sq        mean_sq            F  \
C(School)              2.0  996939.237147  498469.618573  5721.620556   
C(Grade)               2.0     324.472829     162.236415     1.862210   
C(School):C(Grade)     4.0     170.851257      42.712814     0.490274   
Residual            2991.0  260576.984204      87.120356          NaN   

                      PR(>F)  
C(School)           0.000000  
C(Grade)            0.155509  
C(School):C(Grade)  0.742912  
Residual                 NaN  


- PR(>F)로부터 P-value 가 크다면 통계적으로 유의미 하지 않다

In [62]:
# two way ols 모형객체를 생성(교호작용x)
two_way=ols('Final~C(School)+C(Grade)',data=df).fit()

# anova_lm 함수를 사용해 이원배치 분산분석
result=anova_lm(two_way)
print(result)

               df         sum_sq        mean_sq            F    PR(>F)
C(School)     2.0  996939.237147  498469.618573  5725.518315  0.000000
C(Grade)      2.0     324.472829     162.236415     1.863479  0.155312
Residual   2995.0  260747.835461      87.061047          NaN       NaN
